<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/geemap-and-ipyleaflet.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/geemap-and-ipyleaflet.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/geemap/master?filepath=examples/geemap-and-ipyleaflet.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/geemap-and-ipyleaflet.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install the gee Python package

In [1]:
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

## Create an interactive map

In [2]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_minimap(position='bottomright')
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Add tile layers

For example, you can Google Map tile layer:

In [3]:
url = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Map', attribution='Google')

Add Google Terrain tile layer:

In [4]:
url = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Terrain', attribution='Google')

## Add WMS layers
More WMS layers can be found at <https://viewer.nationalmap.gov/services/>.

For example, you can add NAIP imagery.

In [5]:
url = 'https://services.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?'
Map.add_wms_layer(url=url, layers='0', name='NAIP Imagery', format='image/png')

Add USGS 3DEP Elevation Dataset

In [6]:
url = 'https://elevation.nationalmap.gov/arcgis/services/3DEPElevation/ImageServer/WMSServer?'
Map.add_wms_layer(url=url, layers='3DEPElevation:None', name='3DEP Elevation', format='image/png')

## Capture user inputs

In [7]:
import geemap
from ipywidgets import Label
from ipyleaflet import Marker

Map = geemap.Map(center=(40, -100), zoom=4)

label = Label()
display(label)

coordinates = []

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'mousemove':
        label.value = str(latlon)
    elif kwargs.get('type') == 'click':
        coordinates.append(latlon)
        Map.add_layer(Marker(location=latlon))

Map.on_interaction(handle_interaction)

Map

Label(value='')

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [8]:
print(coordinates)

[[43.37336174359601, -103.09356964319335], [41.78795299288423, -104.30230108184963], [41.47591742992624, -97.46747421962957], [35.960778755287464, -99.04736342982633], [38.754618747328976, -110.51932362943685]]


## A simpler way for capturing user inputs

In [9]:
import geemap
Map = geemap.Map(center=(40, -100), zoom=4)
cluster = Map.listening(event='click', add_marker=True)
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [10]:
# Get the last mouse clicked coordinates
Map.last_click

[36.76529191711624, -104.9820602104982]

In [11]:
# Get all the mouse clicked coordinates
Map.all_clicks

[[41.795888098191426, -97.11431739160822],
 [38.02213147353745, -96.05942449969002],
 [39.32579941789298, -112.45421819491885],
 [36.76529191711624, -104.9820602104982]]

## SplitMap control

In [12]:
import geemap
from ipyleaflet import *

Map = geemap.Map(center=(47.50, -101), zoom=7)

right_layer = WMSLayer(
    url = 'https://ndgishub.nd.gov/arcgis/services/Imagery/AerialImage_ND_2017_CIR/ImageServer/WMSServer?',
    layers = 'AerialImage_ND_2017_CIR',
    name = 'AerialImage_ND_2017_CIR',
    format = 'image/png'
)

left_layer = WMSLayer(
    url = 'https://ndgishub.nd.gov/arcgis/services/Imagery/AerialImage_ND_2018_CIR/ImageServer/WMSServer?',
    layers = 'AerialImage_ND_2018_CIR',
    name = 'AerialImage_ND_2018_CIR',
    format = 'image/png'
)

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
Map.add_control(control)
Map.add_control(LayersControl(position='topright'))
Map.add_control(FullScreenControl())

Map

Map(center=[47.5, -101], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…